#### Package installations
Uncomment if use GPU rent services

In [1]:
# !pip3 install torch==1.9.0+cu111 torchvision==0.10.0+cu111 torchaudio===0.9.0 -f https://download.pytorch.org/whl/torch_stable.html

In [2]:
# !pip install nuscenes-devkit

In [3]:
# # Fix error : libGL.so.1: cannot open shared object file: No such file or directory
# !apt install -y libgl1-mesa-glx

In [1]:
import torch

print(torch.cuda.is_available())

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Running on ', device)
print(f"Device count: {torch.cuda.device_count()}")

True
Running on  cuda:0
Device count: 1


In [2]:
import os
import zipfile

CURRENT_PATH = f'{os.getcwd()}/'

#### Unzip nuScenes dataset

In [3]:
# DatasetName = 'Dataset.zip'
#
# print(f'{CURRENT_PATH}{DatasetName}')
#
# with zipfile.ZipFile(DatasetName, 'r') as zip_ref:
#     zip_ref.extractall(f'{CURRENT_PATH}{DatasetName}')

#### NuScenes initialization

In [4]:
from nuscenes import NuScenes
from nuscenes.prediction import PredictHelper
from nuscenes.eval.prediction.splits import get_prediction_challenge_split

import matplotlib.pyplot as plt

# This is the path where you stored your copy of the nuScenes dataset.
DATAROOT = 'Dataset/'

history_length = 2
prediction_length = 6

# Use v1.0-trainval or v1.0-mini
nusc = NuScenes('v1.0-trainval', dataroot=DATAROOT, verbose=False)
helper = PredictHelper(nusc)

In [5]:
train = get_prediction_challenge_split("train", dataroot=DATAROOT)
validation = get_prediction_challenge_split("train_val", dataroot=DATAROOT)
test = get_prediction_challenge_split("val", dataroot=DATAROOT)

print(f"Train len: {len(train)}\nVal len: {len(validation)}\nTest len: {len(test)}")

Train len: 32186
Val len: 8560
Test len: 9041


In [6]:
train = train[:12000]
validation = validation[:5000]
test = test[:5000]

In [7]:
import pickle

PATH_TO_EPSILON_8_SET = f"{DATAROOT}prediction_trajectory_sets/epsilon_8.pkl"
trajectories_set_8 = pickle.load(open(PATH_TO_EPSILON_8_SET, 'rb'))
trajectories_set_8 = torch.Tensor(trajectories_set_8)

#### Init datasets and dataloaders

In [8]:
from torch.utils.data import DataLoader, Dataset

from nuscenes.prediction.input_representation.static_layers import StaticLayerRasterizer

import numpy as np
from typing import List

class NuscenesDataset(Dataset):
    def __init__(self, tokens: List[str], helper: PredictHelper):
        self.tokens = tokens
        self.static_layer_representation = StaticLayerRasterizer(helper)

    def __len__(self):
        return len(self.tokens)
    
    def __getitem__(self, index: int):

        token = self.tokens[index]
        instance_token, sample_token = token.split("_")

        image = self.static_layer_representation.make_representation(instance_token, sample_token)
        image = torch.Tensor(image).permute(2, 0, 1)

        # NaN Values processing
        def agent_param_processing(value):
            if np.isnan(value):
                return -1
            return value
        
        vel = helper.get_velocity_for_agent(instance_token, sample_token)
        vel = agent_param_processing(vel)
        
        accel = helper.get_acceleration_for_agent(instance_token, sample_token)
        accel = agent_param_processing(accel)
        
        heading_cr = helper.get_heading_change_rate_for_agent(instance_token, sample_token)
        heading_cr = agent_param_processing(heading_cr)
                
        agent_state_vector = torch.Tensor([vel, accel, heading_cr])

        ground_truth = helper.get_future_for_agent(instance_token, sample_token, prediction_length, in_agent_frame=True)

        # Convert to [batch_size, 1, 12, 2]
        # Because loss function need that format
        ground_truth = np.expand_dims(ground_truth, 0)

        return image, agent_state_vector, ground_truth

In [9]:
batch_size = 32

train_ds = NuscenesDataset(train, helper)
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)

train_val_ds = NuscenesDataset(validation, helper)
train_val_dl = DataLoader(train_ds, batch_size=batch_size * 2)

In [10]:
image, state, ground_truth = next(iter(train_dl))
print(image.size())
print(state.size())
print(ground_truth.size())

print("Preprocessing states:")
print(state)

torch.Size([32, 3, 500, 500])
torch.Size([32, 3])
torch.Size([32, 1, 12, 2])
Preprocessing states:
tensor([[ 6.5560e+00, -7.0714e-01,  7.7550e-02],
        [ 1.0361e+01,  1.4954e-04,  0.0000e+00],
        [ 3.8700e+00, -1.0000e+00, -1.9387e-01],
        [ 3.8800e+00,  2.4926e-01,  0.0000e+00],
        [ 1.8056e-01,  6.5437e-04,  1.3966e-02],
        [ 1.1278e+01,  2.4033e-03,  0.0000e+00],
        [ 7.4143e+00, -1.0000e+00,  1.8384e-02],
        [ 5.7498e+00, -5.7486e-01,  0.0000e+00],
        [ 9.9595e+00, -6.1523e-01,  7.7553e-02],
        [ 2.1336e+00, -4.8064e-02, -3.0270e-01],
        [ 5.0137e-01, -5.0147e-03,  0.0000e+00],
        [ 9.2956e+00, -1.6963e+00,  0.0000e+00],
        [ 1.6924e+00, -3.6643e-04,  1.6621e-02],
        [ 3.3864e+00,  1.2458e+00,  4.2652e-01],
        [ 2.4337e-02,  3.0541e-03,  2.1503e-02],
        [ 3.7095e+00, -9.4885e-04,  2.6958e-01],
        [ 3.3353e+00,  3.8862e-03,  2.5437e-01],
        [ 7.2877e+00,  5.9953e+00, -2.9668e-01],
        [ 1.2844e+0

#### Init ML prediction model

In [11]:
from nuscenes.prediction.models.backbone import ResNetBackbone
import torchvision.models as models

# Torchvision backbone
backbone = models.resnext50_32x4d(pretrained=True)

# Build-in backbone
#backbone = ResNetBackbone('resnet50')

# Set backbone to non-trainable
def set_parameter_requires_grad(model):
    for param in model.parameters():
        param.requires_grad = False
        
set_parameter_requires_grad(backbone)

In [12]:
from torch.optim import SGD
from nuscenes.prediction.models.covernet import CoverNet, ConstantLatticeLoss

NUM_MODES = 64

model = CoverNet(backbone, num_modes=NUM_MODES)
model = model.to(device)

loss_function = ConstantLatticeLoss(trajectories_set_8)

c:\users\impor\appdata\local\programs\python\python37\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


In [13]:
# Pass to optimizer only params with requires_grad
params_to_update = []

for name,param in model.named_parameters():
    if param.requires_grad == True:
        params_to_update.append(param)
        print("\t",name)

optimizer = SGD(params_to_update, lr=5e-4, momentum=0.9, weight_decay=5e-4)

	 head.0.weight
	 head.0.bias
	 head.1.weight
	 head.1.bias


In [14]:
from tqdm import tqdm
import copy
import time

def loss_batch(model, loss_func, img, state_vec, ground_truth, opt=None):
    img = img.to(device)
    state_vec = state_vec.to(device)
    ground_truth = ground_truth.to(device)
    
    predicted_logits = model(img, state_vec)
    loss = loss_func(predicted_logits, ground_truth)

    # For validation optimizer is None, thus we dont perform backprop
    if opt is not None:
        loss.backward()
        opt.step()
        opt.zero_grad()

    # Return losses and amount of items
    # print(f"{loss.item()}; {len(img)}")
    return loss.item(), len(img)


def fit(epochs, model, loss_func, opt, train_dl, valid_dl):
    best_loss = 999.0
    best_model_wts = copy.deepcopy(model.state_dict())

    for epoch in range(epochs):
        start_epoch_time = time.time()
        print(f'Epoch: {epoch + 1}/{epochs}')
        print('-' * 10)
        
        model.train()

        for img, state_vec, gt in tqdm(train_dl):
            loss_batch(model, loss_func, img, state_vec, gt, opt)

        model.eval()
        print("Validation step")

        with torch.no_grad():
            # TODO: Using tqdm
            losses, nums = zip(
                *[loss_batch(model, loss_func, img, state_vec, gt) for img, state_vec, gt in valid_dl]
            )

        val_loss = np.sum(np.multiply(losses, nums)) / np.sum(nums)
        
        # deep copy the model
        if val_loss < best_loss:
            best_loss = val_loss
            best_model_wts = copy.deepcopy(model.state_dict())

        print(f"Epoch {epoch + 1}; Loss: {val_loss:0.2f}; Best: {best_loss:0.2f} Time: {(time.time() - start_epoch_time):0.2f} sec;")


In [15]:
# epochs = 3  # how many epochs to train for
# fit(epochs, model, loss_function, optimizer, train_dl, train_val_dl)

In [16]:
# torch.save(model.state_dict(), '/root/model.pth')

In [17]:
model.load_state_dict(torch.load('./Models/model_data_5000_e25_loss_1-15.pth'))
model.eval()

CoverNet(
  (backbone): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
        (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Seq

#### Metrics

In [18]:
NPY_DATA_PATH = './NpyDataset/'

test_features = np.load(f'{NPY_DATA_PATH}data_test_features_5k.npy')
test_states = np.load(f'{NPY_DATA_PATH}data_test_states_5k.npy')
test_labels = np.load(f'{NPY_DATA_PATH}data_test_labels_5k.npy')

In [19]:
def PlotPrediction(future, predict):
  plt.figure(figsize=(6, 6))

  plt.scatter(future[:, 1], -future[:, 0], c='orange', s=10)
  plt.scatter(predict[:, 1], -predict[:, 0], c='g', s=10)

  # Keep aspect ratio of axis
  plt.axis('equal')
  plt.show()


In [23]:
import nuscenes.eval.prediction.metrics as metrics

trajectories_set_8_np = trajectories_set_8.numpy()

minFDE_k = metrics.MinFDEK([1, 5], aggregators=[metrics.RowMean()])

metrics_container = []

for idx in range(5):
    # Make prediction
    img = torch.Tensor(test_features[idx].reshape((500, 500, 3))).permute(2, 0, 1).unsqueeze(0)
    img = img.to(device)

    state = torch.Tensor(np.array([test_states[idx]])).to(device)
    state = state.to(device)

    logits = model(img, state)
    mode_probabilities = np.array([logits.cpu().detach().numpy()[0]])[0]

    # Create prediction object
    instance_tkn, sample_tkn = test[idx].split("_")
    prediction = metrics.Prediction(instance_tkn, sample_tkn, trajectories_set_8_np, mode_probabilities)

    # Get ground_truth
    gt = test_labels[idx].reshape((12, 2))

    # Calculate metrics
    fde = minFDE_k(gt, prediction)
    metrics_container.append(fde)
    print(fde)

[[4.35029012 4.35029012]]
[[3.67147222 3.67147222]]
[[3.76456152 3.02057887]]
[[3.24151024 2.26132469]]
[[3.72568621 2.34562331]]


In [26]:
print(metrics_container)

for agg in minFDE_k.aggregators:
    a = agg(np.array(metrics_container))
    print(a)

[array([[4.35029012, 4.35029012]]), array([[3.67147222, 3.67147222]]), array([[3.76456152, 3.02057887]]), array([[3.24151024, 2.26132469]]), array([[3.72568621, 2.34562331]])]
[[3.7507040602032733, 3.1298578419185428]]


In [27]:
stacked_trajs = trajectories_set_8.numpy()

min_ade_k = 5

total_min_fde = 0
total_min_ade = 0

#MissRateTopK = metrics.MissRateTopK([5, 10], tolerance=2, aggregators=[metrics.RowMean()])

#for idx in range(len(test_labels)):
for idx in range(5):
    img = torch.Tensor(test_features[idx].reshape((500, 500, 3))).permute(2, 0, 1).unsqueeze(0)
    img = img.to(device)

    state = torch.Tensor(np.array([test_states[idx]])).to(device)
    state = state.to(device)

    # print(img.size())
    # print(state.size())

    logits = model(img, state)
    mode_probabilities = np.array([logits.cpu().detach().numpy()[0]])

    # TODO: Use metrics.stack_ground_truth
    N = 64
    gt = test_labels[idx].reshape((12, 2))
    stacked_ground_truth = np.empty((N, 12, 2))
    for i in range(N):
        stacked_ground_truth[i] = gt

    #min_fde_k = 1
    min_fde = metrics.min_fde_k(stacked_trajs, stacked_ground_truth, mode_probabilities)
    min_ade = metrics.min_ade_k(stacked_trajs, stacked_ground_truth, mode_probabilities)
    miss_rate = metrics.MissRateTopK

    min_fde = min_fde[0][0]
    min_ade = min(min_ade[0][:min_ade_k])

    # Plot ground_truth and Top1 trajectory
    # sorted_logits_indexes = logits.argsort(descending=True)
    # sorted_trajectories = trajectories_set_8[sorted_logits_indexes][0]

    # for i in range(min_ade_k):
    #     PlotPrediction(gt, sorted_trajectories[i])

    print(f"[{idx}]: mFDE: {min_fde}; mADE: {min_ade}")
    total_min_fde += min_fde
    total_min_ade += min_ade


[0]: mFDE: 4.350290120077532; mADE: 3.550774972838594
[1]: mFDE: 3.6714722223046516; mADE: 3.337161391862899
[2]: mFDE: 3.764561515284531; mADE: 3.055059685830171
[3]: mFDE: 3.2415102377064393; mADE: 2.7349237477942556
[4]: mFDE: 3.7256862056432123; mADE: 3.062043088546279


In [29]:
n = 5 # len(test_labels)

total_min_fde / n, total_min_ade / n

(3.7507040602032733, 3.1479925773744393)

In [22]:
instance_token, sample_token = test[0].split('_')

In [23]:
mode_probabilities

array([[ 5.83557606e+00,  3.56018233e+00, -4.36437654e+00,
         4.93365765e+00, -1.11988049e+01, -1.22068195e+01,
         5.44546318e+00,  3.47506762e+00,  2.52648759e+00,
         6.95708275e+00,  1.13715184e+00,  2.36022234e+00,
        -7.45455742e+00,  5.44341707e+00,  2.45202971e+00,
        -2.93625879e+00, -4.13234532e-03,  3.38360047e+00,
        -1.07468967e+01, -8.43985844e+00,  8.25137734e-01,
         1.98839319e+00,  5.07631826e+00,  2.12939382e+00,
         5.12734509e+00,  3.80032206e+00,  2.09863448e+00,
         8.32696795e-01, -4.28695679e+00, -1.23020220e+01,
         1.78788447e+00,  1.03175819e+00, -5.53425407e+00,
         2.09780192e+00, -5.24218607e+00, -3.53525591e+00,
        -4.80795860e+00,  4.16410780e+00,  2.38868237e+00,
        -2.16933298e+00,  9.68165934e-01, -3.20030141e+00,
         4.50264883e+00,  2.16203046e+00, -2.34711123e+00,
         3.72334504e+00,  3.80144089e-01,  3.40158916e+00,
        -7.92281091e-01, -2.27562451e+00,  3.01750159e+0

In [31]:
prediction = metrics.Prediction(instance_token, sample_token, trajectories_set_8.numpy(), mode_probabilities[0])

minFDE_k = metrics.MinFDEK([1, 5], aggregators=[metrics.RowMean()])
minADE_k = metrics.MinADEK([5, 10], aggregators=[metrics.RowMean()])
missRate_k = metrics.MissRateTopK([5, 10], tolerance=2, aggregators=[metrics.RowMean()])

fde = minFDE_k(gt, prediction)
print(fde)

ade = minADE_k(gt, prediction)
print(ade)

miss_rate = missRate_k(gt, prediction)
print(miss_rate)

[[4.35029012 4.35029012]]
[[3.55077497 3.55077497]]
[[ True  True]]


In [ ]:
# import nuscenes.eval.prediction.metrics as metrics
#
# a = metrics.min_fde_k(stacked_trajs, stacked_ground_truth, mode_probabilities)
# a


In [ ]:
# from nuscenes.eval.prediction.tests.test_metrics import TestFunctions

# tf = TestFunctions()

# tf.setUp()
# tf.test_min_fde_k_many_batches_and_modes()